<a href="https://colab.research.google.com/github/vaseem-20/Artificial-Intelligence/blob/main/Language_Identifier_APP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

In [101]:
data = pd.read_csv('/content/Language_Detection_Dataset.csv')

In [102]:
data.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


In [103]:
len(data.language.value_counts())

22

In [104]:
x_train,x_test,y_train,y_test = train_test_split(data.Text,data.language,test_size=0.2,random_state=42)

In [105]:
lable= LabelEncoder()
y_train_el = lable.fit_transform(y_train)
y_test_el = lable.fit_transform(y_test)

In [106]:
len(set(y_test_el))

22

In [125]:
from string import punctuation
def preprocces(text):
  data=word_tokenize(text)
  data=[i for i in data if i not in punctuation]
  data=" ".join(data)
  return data

In [108]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [109]:
x_train_final=x_train.apply(preprocces)
x_test_final=x_test.apply(preprocces)

In [110]:
x_train_final

,Text
5207,สัมประสิทธิ์ฮอลล์ ฟิสิกส์ไฟฟ้า เกี่ยวกับสนามแม...
4450,เกิดวันที่ พฤศจิกายน ภาคอะนิเมะ คดีฆาตกรรมบนจอ...
7033,i omgivningarna runt manigotagan river park re...
487,நிஞ்சா ஹட்டோரி 忍者ハットリくん ninja hattori என்பது க...
19537,эта страница деятельности м в ломоносова — ярк...
...,...
11964,باباجان غفورف تاریخ‌دان و نویسندهٔ کتاب تاریخ ...
21575,en fue invitado por fernando ii para ocupar la...
5390,doğu kanada atabasklarına geleneksel olarak dü...
860,پژواک د يوې ځانگړې پروژې په توگه د اساسي قانون...


In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [112]:
tf=TfidfVectorizer(lowercase=True,stop_words='english',max_df=0.95,max_features=1000)
tf_train=tf.fit_transform(x_train_final).toarray()
tf_test=tf.transform(x_test_final).toarray()

In [113]:
from sklearn.naive_bayes import MultinomialNB

MultinomialNB()

In [115]:
y_pred=model.predict(tf_test)
y_pred

array([17, 15, 10, ...,  3, 13,  2])

In [116]:
from sklearn.metrics import accuracy_score

In [117]:
acc = accuracy_score(y_test_el,y_pred)
acc

0.8825

In [118]:
def predict_lang(text):
  x = preprocces(text)
  data=tf.transform([x]).toarray()
  lang=model.predict(data)
  lang=lable.inverse_transform(lang)
  return lang[0]

In [119]:
predict_lang('Me llamo Mondly')

'Swedish'

In [120]:
predict_lang('Hello ')

'Swedish'

In [128]:
!pip install gradio

In [138]:
import gradio as gr
import pandas as pd
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import nltk
import warnings
warnings.simplefilter('ignore')
# download required NLTK resources
nltk.download('punkt')
# load dataset
data = pd.read_csv('/content/Language_Detection_Dataset.csv')
# Splitting the dataaet into training and testing
x_train,x_test,y_train,y_test = train_test_split(data.Text,data.language,test_size=0.2,random_state=42)
# Encode labels into numeric Values
lable= LabelEncoder()
y_train_el = lable.fit_transform(y_train)
y_test_el = lable.fit_transform(y_test)
# text preprocessing function
def preprocces(text):
  data=word_tokenize(text)
  data=[i for i in data if i not in punctuation]
  return" ".join(data)
# Applying preproccesing to training testing text data
x_train_processed=x_train.apply(preprocces)
x_test_processed=x_test.apply(preprocces)
# IF-IDE Vectorization
vectorizer=TfidfVectorizer(lowercase=True,stop_words='english',max_df=0.95,max_features=1000)
tf_train=vectorizer.fit_transform(x_train_processed).toarray()
tf_test=vectorizer.transform(x_test_processed).toarray()
# Train Naive Bayes Model
model=MultinomialNB()
model.fit(tf_train,y_train_el)
# Predict function for Gradio
def predict_lang(text):
  x = preprocces(text)
  data=vectorizer.transform([x]).toarray()
  lang=model.predict(data)
  lang=lable.inverse_transform(lang)
  return f"Predict Language: {lang[0]}"

# Gradio Interface
iface=gr.Interface(
    fn=predict_lang,
    inputs=gr.Textbox(placeholder='Enter a Sentence...'),
    outputs=gr.Textbox(),
    title='Language Identifier',
    description='Identify the language of a given text.',
)

# Launch Gradio App
iface.launch(share=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8233929b6554e3056.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
